In [1]:
import tensorflow as tf
import os
import keras.backend as K
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model
from dataset import *

task_names = ['skirt_length', 'coat_length', 'collar_design', 'neck_design', 'neckline_design', 'pant_length', 'sleeve_length', 'lapel_design']



Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

In [3]:
task_name = task_names[7]
df = pd.read_csv('data/Annotations/label.csv', header=None)
df.columns = ['filename', 'label_name', 'label']

df.label_name = df.label_name.str.replace('_labels', '')
df = df[df.label_name == task_name]
df = df.sample(frac=1).reset_index(drop=True) # shuffle

c = Counter(df.label_name)
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
n = len(df)
fnames = df['filename'].values
width = 399

y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
df = df.reset_index(drop=True)

def f(index):
    if os.path.isfile('data/'+fnames[index]):
        return index, cv2.resize(cv2.imread('data/'+fnames[index]), (width, width))

for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1
    #y[label_names.index(label_name)][i, label.find('m')] = 0.5

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

n_train = int(n*0.8)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

gen_train = Generator(X_train, y_train, batch_size=16, aug=True)

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
base_model = InceptionV3(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling = 'avg')

input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

gpumodel = multi_gpu_model(model, n_gpus)



100%|██████████| 7034/7034 [00:17<00:00, 405.30it/s]


In [4]:
gpumodel.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/3
352/352 [==============================] - 329s 936ms/step - loss: 1.0153 - acc: 0.6006 - val_loss: 0.5750 - val_acc: 0.7882
Epoch 2/3
352/352 [==============================] - 243s 690ms/step - loss: 0.5899 - acc: 0.7923 - val_loss: 0.4813 - val_acc: 0.8401
Epoch 3/3
352/352 [==============================] - 243s 690ms/step - loss: 0.4245 - acc: 0.8602 - val_loss: 0.4164 - val_acc: 0.8515


In [5]:
gpumodel.compile(optimizer=Adam(0.000025), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/2
352/352 [==============================] - 315s 895ms/step - loss: 0.2496 - acc: 0.9202 - val_loss: 0.3956 - val_acc: 0.8756
Epoch 2/2
352/352 [==============================] - 234s 666ms/step - loss: 0.1777 - acc: 0.9449 - val_loss: 0.4162 - val_acc: 0.8692


In [6]:
gpumodel.compile(optimizer=Adam(0.00000625), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/3
352/352 [==============================] - 330s 938ms/step - loss: 0.1412 - acc: 0.9562 - val_loss: 0.3684 - val_acc: 0.8849
Epoch 2/3
352/352 [==============================] - 237s 674ms/step - loss: 0.1283 - acc: 0.9623 - val_loss: 0.3631 - val_acc: 0.8870
Epoch 3/3
352/352 [==============================] - 237s 672ms/step - loss: 0.1085 - acc: 0.9687 - val_loss: 0.3720 - val_acc: 0.8934


In [7]:
gpumodel.compile(optimizer=Adam(0.00000425), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)

Epoch 1/1
352/352 [==============================] - 334s 949ms/step - loss: 0.1021 - acc: 0.9699 - val_loss: 0.3742 - val_acc: 0.8934


In [8]:
gpumodel.compile(optimizer=Adam(0.000001), loss='categorical_crossentropy', metrics=['accuracy'])
gpumodel.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)


Epoch 1/1
352/352 [==============================] - 346s 982ms/step - loss: 0.1006 - acc: 0.9673 - val_loss: 0.3605 - val_acc: 0.8927


In [9]:
model.compile(optimizer=SGD(lr=1e-8, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

model_name = 'InceptionV3' + task_name
model.save('model_%s.h5' % model_name)


Epoch 1/1
352/352 [==============================] - 441s 1s/step - loss: 0.1036 - acc: 0.9641 - val_loss: 0.3574 - val_acc: 0.8927
